In [1]:
%load_ext autoreload
%autoreload 2
import os
from Pytorch_VAE_LSTM import LSTMVAE
from music_dataset import TorchMusicDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
import numpy as np

import scipy.io.wavfile as wavfile

In [2]:
epochs = 1
batch_size = 4
hidden_dim = 100
latent_dim = 100
n_seconds = 10

In [3]:
# 0 elektronik - 1 folk
song_classes = [0 for i in range(50)] + [1 for i in range(50)]
songs_path = "songs"

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
dataset = TorchMusicDataset("../cleaned_data", n_seconds)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
n_timesteps, n_features  = dataset.__getitem__(1).shape[:2]
n_timesteps, n_features

(44100, 11)

In [6]:
dataset.__getitem__(1).shape

torch.Size([44100, 11])

In [7]:
vae = LSTMVAE(n_timesteps, n_features, hidden_dim, latent_dim, device).to(device)
vae.optimizer = optim.Adam(vae.parameters(), lr=1e-3)

In [8]:
vae.load_state_dict(torch.load("model.torch"))

/tmp/ipykernel_189906/2509860592.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load("model.torch"))


<All keys matched successfully>

In [9]:
if not os.path.exists(songs_path):
    os.mkdir(songs_path)

In [10]:
for i, c in enumerate(song_classes):
    song = vae.generate_sequences([c])[0].reshape(-1)
    wavfile.write(
        f"{songs_path}/{i}.wav",
        44100,
        song
    )